In [1]:
import pandas as pd
import numpy as np

In [2]:
file = "pkl/uber-raw-data-combined.csv.pkl"
df = pd.read_pickle(file)
df

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512
...,...,...,...,...
4534322,9/30/2014 22:57:00,40.7668,-73.9845,B02764
4534323,9/30/2014 22:57:00,40.6911,-74.1773,B02764
4534324,9/30/2014 22:58:00,40.8519,-73.9319,B02764
4534325,9/30/2014 22:58:00,40.7081,-74.0066,B02764


In [3]:
def time_sampling(start, end):
    df["Date/Time"] = pd.to_datetime(df["Date/Time"])
    filter_df = df[(df["Date/Time"] >= start) & (df["Date/Time"] <= end)]
    sample_df = filter_df.sort_values(by="Date/Time")
    return sample_df

### Inputs go by:
"Year-Month-Day Hour:Minute:Second"

*time is in 24 hour clock

the rows are out of order because original csv was sorted by base values, not time.

In [4]:
#time_sampling("2014-04-01 00:00:00", "2014-04-29 00:00:00")

In [5]:
if (df["Date/Time"].dtypes != np.datetime64):
    df["Date/Time"] = pd.to_datetime(df["Date/Time"])
print(f"Max: {df.max()}")
print(f"Min: {df.min()}")
lat = df["Lat"].sort_values().unique()
lon = df["Lon"].sort_values().unique()
time = df["Date/Time"].sort_values().unique()

Max: Date/Time    2014-09-30 22:59:00
Lat                      42.1166
Lon                     -72.0666
Base                      B02764
dtype: object
Min: Date/Time    2014-04-01 00:00:00
Lat                      39.6569
Lon                      -74.929
Base                      B02512
dtype: object


In [21]:
def cluster(df, bin):
    tf = pd.cut(df, bins=bin)
    count = tf.value_counts().sort_index() #datatype = pandas series
    print(count) #.index[0] for range, .iloc[0] for value

In [25]:
cluster(time, 1000)

(2014-03-31 19:36:32.460000, 2014-04-01 04:23:27.540000]    193
(2014-04-01 04:23:27.540000, 2014-04-01 08:46:55.080000]    261
(2014-04-01 08:46:55.080000, 2014-04-01 13:10:22.620000]    264
(2014-04-01 13:10:22.620000, 2014-04-01 17:33:50.160000]    263
(2014-04-01 17:33:50.160000, 2014-04-01 21:57:17.700000]    246
                                                           ... 
(2014-09-30 01:01:42.300000, 2014-09-30 05:25:09.840000]    246
(2014-09-30 05:25:09.840000, 2014-09-30 09:48:37.380000]    263
(2014-09-30 09:48:37.380000, 2014-09-30 14:12:04.920000]    264
(2014-09-30 14:12:04.920000, 2014-09-30 18:35:32.460000]    263
(2014-09-30 18:35:32.460000, 2014-09-30 22:59:00]           264
Name: count, Length: 1000, dtype: int64


In [28]:
cluster(lat, 100)

(39.654, 39.681]    2
(39.681, 39.706]    0
(39.706, 39.731]    1
(39.731, 39.755]    0
(39.755, 39.78]     0
                   ..
(41.994, 42.018]    0
(42.018, 42.043]    0
(42.043, 42.067]    0
(42.067, 42.092]    0
(42.092, 42.117]    1
Name: count, Length: 100, dtype: int64


In [29]:
cluster(lon, 100)

(-74.932, -74.9]      1
(-74.9, -74.872]      0
(-74.872, -74.843]    2
(-74.843, -74.815]    3
(-74.815, -74.786]    0
                     ..
(-72.21, -72.181]     0
(-72.181, -72.152]    1
(-72.152, -72.124]    0
(-72.124, -72.095]    0
(-72.095, -72.067]    1
Name: count, Length: 100, dtype: int64
